# Deprecated look at geo_metada instead


In [17]:
from Bio import Entrez
import redis
import json

In [18]:
# Downloading data with query: 'single+cell+rna'
Entrez.email = "yunfang@chanzuckerberg.com"
handle = Entrez.esearch(db="gds", retmax=500, retstart=5000, term="single cell rna")
record = Entrez.read(handle)
handle.close()



In [19]:
x = redis_store.get('sr:pg:1')

In [3]:
redis_store = redis.StrictRedis(host='localhost', port=6379, db=2)
SR_PREFIX = 'sr:pg:'
PAGE_SIZE = 500


In [ ]:
#redis_store.set('sdfdf','sdfdf')
import time

page = 111
page_size = PAGE_SIZE
while page_size >= PAGE_SIZE:
    print ("Downloading page %d" % page)
    handle = Entrez.esearch(db="gds", retmax=PAGE_SIZE, retstart=page*PAGE_SIZE, term="single cell rna")
    record = Entrez.read(handle)
    handle.close()
    json_str = json.dumps(record['IdList'])
    redis_key = "%s%d" %(SR_PREFIX, page)
    redis_store.set(redis_key, json_str)
    page_size = len(record['IdList'])
    page+=1
    time.sleep(2)



In [64]:
Entrez.email = "yunfang@chanzuckerberg.com"

PAGE_SIZE = 500
def fetch_search_results(query, mydb, pfx, page=0):
    page_size = PAGE_SIZE
    while page_size >= PAGE_SIZE:
        print("Downloading page %d for %s" % (page, query))
        try:
            handle = Entrez.esearch(db=mydb, retmax=PAGE_SIZE, retstart=page*PAGE_SIZE, term=query)
            record = Entrez.read(handle)
            handle.close()
            json_str = json.dumps(record['IdList'])
            redis_key = "%s%d" %(pfx, page)
            redis_store.set(redis_key, json_str)
            page_size = len(record['IdList'])
            page+=1
            time.sleep(2)
        except Exception:
            print("Error getting page %d" % page)
            time.sleep(5)
# 67835 

In [65]:
fetch_search_results('"single cell" rna','pubmed','sr:pm1:', 0)
fetch_search_results('"single-cell" rna','pubmed','sr:pm2:', 0)

In [ ]:
from Bio import Entrez
import redis
import json

redis_store = redis.StrictRedis(host='localhost', port=6379, db=2)

handle = Entrez.esummary(db='gds', id='200067835')
records = Entrez.read(handle)
handle.close()
for r in records:
    print(r)
    redis_store.set('gds:'+ r['Id'], json.dumps(r))
    

In [23]:
GDS_PREFIX = 'gds:'
def fetch_esummary(id_str, mydb='gds', pfx = GDS_PREFIX):
    while True:
        try:
            handle = Entrez.esummary(db=mydb, id=id_str)
            records = Entrez.read(handle)
            handle.close()
            
            for r in records: 
                key = "%s%s" %(pfx, r['Id'])
                val = json.dumps(r)
                redis_store.set(key, val)
            return
        except Exception:
            print("ERROR getting %s" % id_str)
            time.sleep(5)


In [ ]:
for sre in ['sr:pm1:', 'sr:pm2:']:
    page = 0 
    while page <= 7:
        print "Downloading page %d for %s" % (page, sre)
        redis_key = "%s%d" %(sre, page) 
        res = redis_store.get(redis_key) 
        id_list = json.loads(res)
        id_str = id_list[0]
        for idx in range(1,len(id_list)):
            if idx % 5 == 0:
                print ("  idx: %d key: %s" % (idx, id_str))
                fetch_esummary(id_str, 'pubmed', 'pm:')
                time.sleep(1)
                id_str = id_list[idx]
            else:
                id_str = id_str + ',' + id_list[idx]
        print ("  idx: %d key: %s" % (idx, id_str))
        fetch_esummary(id_str, 'pubmed', 'pm:')
        page += 1
        
    

In [9]:
redis_store.get("sr:pm1:4")

In [86]:
records[0]
x = redis_store.get('pm:24069527')
print(x)
records[0]

KeyError: 0

In [ ]:
# validate the data 
page = 0
while page <= 111:
    redis_key = "%s%d" %(SR_PREFIX, page)
    res = redis_store.get(redis_key)
    #print "key: %s len: %d" % (redis_key, len(res))
    page+=1
    


In [ ]:


page = 111
GDS_PREFIX = 'gds:'
while page <= 111:
    print("Processing page %d" % page)
    redis_key = "%s%d" %(SR_PREFIX, page) 
    res = redis_store.get(redis_key) 
    id_list = json.loads(res)
    id_str = id_list[0]
    for idx in range(1,len(id_list)):
        if idx % 5 == 0:
            print ("  idx: %d key: %s" % (idx, id_str))
            fetch_esummary(id_str)
            time.sleep(1)
            id_str = id_list[idx]
        else:
            id_str = id_str + ',' + id_list[idx]
    print ("  idx: %d key: %s" % (idx, id_str))
    fetch_esummary(id_str)
    page += 1
    

In [3]:
# VEGA (table. visualization)
import pprint 
pp = pprint.PrettyPrinter(indent=4) 

page = 0
nsamples = 0
npubmed_articles = 0
nftp_articles = 0
n_points = 0
n_sra = 0
while page <= 111:
    redis_key = "%s%d" %(SR_PREFIX, page) 
    id_list = json.loads(redis_store.get(redis_key)) 
    n_points += len(id_list)
    for doc_id in id_list:
        redis_key = 'gds:' + doc_id
        rec = json.loads(redis_store.get(redis_key))
        npubmed_articles += len(rec['PubMedIds'])
        nsamples += int(rec['n_samples'])
        if len(rec['ExtRelations']) > 0: 
            n_sra +=1
            #pp.print()
        if len(rec['FTPLink']) > 0:
            nftp_articles += 1 
    page +=1

print ("# GSM samples: %d, # pubmed articles: %d, # articles with FTP: %d, # sra: %d, # data points: %d" %(nsamples, npubmed_articles, nftp_articles, n_sra, n_points))

# GSM samples: 710490, # pubmed articles: 5900, # articles with FTP: 26475, # sra: 21147, # data points: 55787


In [109]:
# randomly generated some data
import random

page = 0
while page <= 111:
    redis_key = "%s%d" %(SR_PREFIX, page) 
    id_list = json.loads(redis_store.get(redis_key))
    selected = random.sample(id_list, 5)
    
    for doc_id in selected:
        redis_key = 'gds:' + doc_id
        rec = json.loads(redis_store.get(redis_key))
        print pp.pprint(rec)
        print "================================================"
    page +=1



{   u'Accession': u'GSE70602',
    u'ExtRelations': [   {   u'RelationType': u'SRA',
                             u'TargetFTPLink': u'ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByStudy/sra/SRP/SRP060/SRP060462/',
                             u'TargetObject': u'SRP060462'}],
    u'FTPLink': u'ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE70nnn/GSE70602/',
    u'GDS': u'',
    u'GEO2R': u'no',
    u'GPL': u'11154',
    u'GSE': u'70602',
    u'Id': u'200070602',
    u'Item': [],
    u'PDAT': u'2016/12/31',
    u'PlatformTaxa': u'',
    u'PlatformTitle': u'',
    u'Projects': [],
    u'PubMedIds': [],
    u'Relations': [],
    u'SSInfo': u'',
    u'Samples': [   {   u'Accession': u'GSM1811426',
                        u'Title': u'A498 lincRNA KD by siRNA1'},
                    {   u'Accession': u'GSM1811429',
                        u'Title': u'786-O NTC rep2'},
                    {   u'Accession': u'GSM1811424', u'Title': u'A498 NTC rep1'},
                    {   u'Accession': u

In [27]:
# VEGA (table. visualization)
import pprint 
import re
pp = pprint.PrettyPrinter(indent=4) 

page = 0
sra_list = []
single_cell_list = [] 

sc_re = re.compile("single[\s\-]*cell", re.M)
while page <= 111:
    redis_key = "%s%d" %(SR_PREFIX, page) 
    id_list = json.loads(redis_store.get(redis_key)) 
    
    for doc_id in id_list:
        redis_key = 'gds:' + doc_id
        rec = json.loads(redis_store.get(redis_key))
        
        #if len(rec['FTPLink']) > 0 and len(rec['ExtRelations']) > 0:
        if len(rec['ExtRelations']) > 0:
            sra_list.append(doc_id)
        summary = rec['summary'].lower() 
        title = rec['title'].lower()
        if sc_re.match(summary) or sc_re.match(title):
            single_cell_list.append(doc_id)
    page +=1


In [28]:
print(len(sra_list))
print(len(single_cell_list))
import codecs 
intersected = list(set(sra_list) & set(single_cell_list))
print(len(intersected))
idx = 1; 
nsamples = 0
final_download_list = []
#outf = codecs.open('data_without_pubmed.txt','w', encoding='utf-8')
for doc_id in intersected:
    redis_key = 'gds:' + doc_id
    rec = json.loads(redis_store.get(redis_key))
    if len(rec['PubMedIds']) == 0:
        continue
    final_download_list.append(doc_id)
    pubmed_str = ",".join([str(x) for x in rec['PubMedIds']])
    nsamples += rec['n_samples']
    x = "%d. Doc ID: %s, GSE: %s: PubMed Id: %s, Taxon: %s, Date: %s, Samples: %s\n" % (idx, doc_id, str(rec['GSE']), pubmed_str, rec['taxon'], rec['PDAT'], str(rec['n_samples'])) 
    
    x += "Title: %s \n" % rec['title']
    x += "Summary: %s \n" % rec['summary']
    x += json.dumps(rec['ExtRelations']) + "\n"
    
    #pp.pprint(rec)
    x += "================================================\n"
    print x
    #outf.write(x)
    idx +=1
print "Total # Samples: %d" % nsamples

21147
3605
3001
1. Doc ID: 200052529, GSE: 52529: PubMed Id: 24658644, Taxon: Homo sapiens, Date: 2014/03/23, Samples: 384
Title: Pseudo-temporal ordering of individual cells reveals regulators of differentiation 
Summary: Single-cell expression profiling by RNA-Seq promises to exploit cell-to-cell variation in gene expression to reveal regulatory circuitry governing cell differentiation and other biological processes. Here, we describe Monocle, a novel unsupervised algorithm for ordering cells by progress through differentiation that dramatically increases temporal resolution of expression measurements. This reordering unmasks switch-like changes in expression of key regulatory factors, reveals sequentially organized waves of gene regulation, and exposes regulators of cell differentiation. A functional screen confirms that a number of these regulators dramatically alter the efficiency of myoblast differentiation, demonstrating that single-cell expression analysis with Monocle can unco

In [24]:
# Extracting files information. Estimate Data Size
# http://stackoverflow.com/questions/237699/how-do-i-parse-a-listing-of-files-to-get-just-the-filenames-in-python
import re
from ftplib import FTP 
def parse_ftp(line):
    '''Output is a triple of (filename, type, size)'''
    f_type = line[0]
    splitted = re.split("[\t\s]+", line)
    f_name = splitted[-1]
    f_size = splitted[4]
    return [f_type, f_name, f_size]
    

def get_file_list_old(starting_dir, ftp, recursive = True):
    ftp.cwd(starting_dir)
    print "  Getting %s" % starting_dir
    flist = []
    result_list = []
    ftp.retrlines('LIST', callback=flist.append) 
    for f in flist:
        parsed = parse_ftp(f)
        if parsed[0] == 'd' and recursive: #directory
            current_dir = starting_dir + '/' + parsed[1]
            current_list = get_file_list(current_dir, ftp)
            result_list = result_list + current_list
        else: 
            file_name = parsed[1]
            parsed[1] = starting_dir + '/' + file_name 
            if recursive == False:
                parsed[1] += '/' + file_name + '.sra'
            result_list.append(parsed)
    return result_list

def get_file_list(starting_dir, ftp):
    ftp.cwd(starting_dir)
    flist = ftp.nlst("*/*")
    results = []
    for f in flist:
        results.append(['-', starting_dir + '/' + f, 0])
    return results
    
def get_sra_files(doc_id):
    redis_key = 'gds:' + doc_id
    rec = json.loads(redis_store.get(redis_key))
    sra_list = rec['ExtRelations']
    result_list = []
    for s in sra_list:
        if s["RelationType"] != 'SRA':
            continue
        sra_url = s["TargetFTPLink"]
        m = re.match("ftp\:\/\/([^\/]+)(\/.*)", sra_url)
        ftp_host = m.group(1)
        ftp_dir  = m.group(2)
        ftph = FTP(ftp_host)
        ftph.login()
        result_list = result_list + get_file_list(ftp_dir, ftph)
        ftph.close()
    return result_list
   


In [ ]:
site = "ftp://ftp-trace.ncbi.nlm.nih.gov/"
flist = get_sra_files('200083139')
res = []
for f in flist:
    #res.append(site+f[1])
    print site+f[1]



In [25]:
sra_hash = {}
doc_list = ['200076381']
for doc_id in doc_list:
    print("=======Downloading SRA for doc %s ====" % doc_id)
    #try:
    res = get_sra_files(doc_id)
    sra_hash[doc_id] = res
    redis_key = 'sra:' + doc_id
    redis_val = json.dumps(res)
    redis_store.set(redis_key, redis_val)
    #except Exception:
    #    print "Trouble downloading SRA for doc %s. Error: %s " % (doc_id, str(Exception))
  
# 200086618, 200070630 doesn't exist
# 200060361 have problem
# restart with 200076381 
# Trouble downloading SRA for doc 200070630 
#results = get_sra_files('200084789')
#for r in results:
#    print r
#200086618 doesn't have values
#200070630 doesn't have values
#200076381 doesn't have values
print final_download_list

=======Downloading SRA for doc 200076381 ====


NameError: name 'final_download_list' is not defined

In [26]:
print final_download_list

NameError: name 'final_download_list' is not defined

In [17]:
#print sra_hash['200060361']
#print len(results)
#print len(final_download_list)
import csv
with open('doc_to_download.csv', 'wb') as csvfile:
    downloadwriter = csv.writer(csvfile, delimiter=',')
    
    for doc_id in final_download_list:
        redis_key = 'sra:' + doc_id
        res = redis_store.get(redis_key)
        rec = json.loads(redis_store.get('gds:' + doc_id))
        pubmed_str = ",".join([str(x) for x in rec['PubMedIds']])
        if res is None: 
            print "%s doesn't have values" % doc_id
        else: 
            res = json.loads(res)
            for line in res:
                downloadwriter.writerow([doc_id, str(rec['GSE']), pubmed_str, rec['taxon'], rec['PDAT']] + line)

#ftph = FTP('ftp-trace.ncbi.nlm.nih.gov') 
#ftph.login()
#results = get_file_list('/sra/sra-instant/reads/ByStudy/sra/SRP/SRP060/SRP060462/', ftph)
#for x in results:
#    print x
# Notes Section
#ftp.cwd('/sra/sra-instant/reads/ByStudy/sra/SRP/SRP060/SRP060462/') 
#dir_info = ftp.retrlines('LIST', callback=flist.append)
#for line in flist:
#print parse_ftp(line)
#print(dir_info)
#ftp.cwd('SRR2089336')
#dir_info = ftp.retrlines('LIST')
#print(dir_info)
#ftp.close()

200086618 doesn't have values
200070630 doesn't have values


In [11]:
# flist for '200076381'
ftp = FTP('ftp-trace.ncbi.nlm.nih.gov') 
ftp.login()
doc_id = '200076381'
file_list = get_file_list('/sra/sra-instant/reads/ByStudy/sra/SRP/SRP067/SRP067844/', ftp, False)
print len(file_list)
print file_list[0]

redis_key = 'sra:' + doc_id
redis_val = json.dumps(file_list)
redis_store.set(redis_key, redis_val)

#results = get_file_list('/sra/sra-instant/reads/ByStudy/sra/SRP/SRP060/SRP060462/', ftph)
#for x in results:
#    print x
# Notes Section
#ftp.cwd('/sra/sra-instant/reads/ByStudy/sra/SRP/SRP060/SRP060462/') 
#dir_info = ftp.retrlines('LIST', callback=flist.append)
#for line in flist:
#print parse_ftp(line)
#print(dir_info)
#ftp.cwd('SRR2089336')
#dir_info = ftp.retrlines('LIST')
#print(dir_info)
#ftp.close()

  Getting /sra/sra-instant/reads/ByStudy/sra/SRP/SRP067/SRP067844/
6665
['d', '/sra/sra-instant/reads/ByStudy/sra/SRP/SRP067/SRP067844//SRR4055063/SRR4055063.sra', '0']


True

In [10]:
redis_store.get("sra:200066217")

'[["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813952/SRR1813952.sra", "1856655752"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813953/SRR1813953.sra", "1026808756"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813954/SRR1813954.sra", "977065988"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813955/SRR1813955.sra", "2272044513"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813956/SRR1813956.sra", "1334855855"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813957/SRR1813957.sra", "1694578012"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813958/SRR1813958.sra", "2148038034"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813959/SRR1813959.sra", "2370752207"], ["-", "/sra/sra-instant/reads/ByStudy/sra/SRP/SRP055/SRP055440//SRR1813960/SRR1813960.sra", "1654822228"]]'

In [ ]:
import GEOparse
gse = GEOparse.get_GEO(filepath="/Users/yf/Downloads/GSE66217_family.soft.gz")
gse.metadata
gse.gsms # look at samples' .metadata
gse.gpls # look at platforms .metadata